In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import itertools
import yaml
import joblib
import dill 
import numpy as np
import scipy as sp 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils import shuffle
import shap

In [ ]:
from utils.utility import load_pickle, read_table_file, save_pickle, save_dataframe_dict
from evaluation.calibration import calibration_test
from training.model_training import run_automl_models


In [ ]:

plt.rcParams['pdf.use14corefonts'] = True
plt.rc('font', family='Helvetica')

plt.style.use('ggplot')
# plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['figure.facecolor'] = 'white'


# Data Loading


In [ ]:
xydata_dir = 'data/cohorts_for_learning/xydata_220913'
dataset_4d_caprini_dict = load_pickle(os.path.join(xydata_dir, f'dataset_4d_scheme3_caprini.pkl'))

task_4d_caprini_df = pd.concat(
    [
        dataset_4d_caprini_dict['x_data'],
        dataset_4d_caprini_dict['y_data'],
        dataset_4d_caprini_dict['misc_data'][['dataset', 'outcome']]
    ],
    axis=1
)

task_4d_caprini_df.loc[task_4d_caprini_df['dataset'] == 'test', 'dataset'] = 'inner_test'


In [ ]:
train_idx = task_4d_caprini_df[task_4d_caprini_df.dataset == 'train'].index
test_idx = task_4d_caprini_df[task_4d_caprini_df.dataset == 'inner_test'].index
caprini_feats = task_4d_caprini_df.columns[:-3]

x_train, y_train = task_4d_caprini_df.loc[train_idx, caprini_feats], task_4d_caprini_df.loc[train_idx, 'label']
x_test, y_test = task_4d_caprini_df.loc[test_idx, caprini_feats], task_4d_caprini_df.loc[test_idx, 'label']
y_train.value_counts(), y_test.value_counts()

# Model training


## Hyper-parameter tuning

In [ ]:

_config_file = 'data/automl/config.yml'
_out_dir = 'test/caprini_automl/1011_1'
save_dir = f"{_out_dir}_additional"

In [ ]:
%%time

run_automl_models(
    task_4d_caprini_df.reset_index(), _config_file, _out_dir,
    label_col='label', dataset_col='dataset', train_name='train', misc_cols=['visit_id', 'outcome']
)


## Feature selection by Wrapping / Regularization


In [ ]:
# Load models from automl tuning
lasso_model_components_dict = joblib.load(
    os.path.join(_out_dir, 'logistic_regression/predict_model_components.pkl')
)
lasso_model = lasso_model_components_dict['model'].model

xgb_model_components_dict = joblib.load(
    os.path.join(_out_dir, 'xgboost/predict_model_components.pkl')
)
xgb_model = xgb_model_components_dict['model'].model

rf_model_components_dict = joblib.load(
    os.path.join(_out_dir, 'random_forest/predict_model_components.pkl')
)
rf_model = rf_model_components_dict['model'].model

print(list(lasso_model_components_dict))
print(list(xgb_model_components_dict))
print(xgb_model.get_params())
print(rf_model.get_params())


### LR by Lasso

In [ ]:

feats_lasso = np.array(lasso_model_components_dict['trained_features'])[lasso_model.coef_.flatten() != 0]
lr0 = LogisticRegression(C=100, max_iter=1000).fit(x_train[feats_lasso], y_train)


### XGB with RFE

In [ ]:
%%time

rfe_xgb1 = RFECV(
    xgb_model,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=2),
    scoring='roc_auc', min_features_to_select=1
)
rfe_xgb1.fit(x_train, y_train)


In [ ]:
# XGB进一步特征选择，提升泛化性，根据RFE做特征选择
_feats_imp = pd.Series(
    rfe_xgb1.estimator_.feature_importances_, index=rfe_xgb1.get_feature_names_out()
) > 0
feats_xgb1 = _feats_imp[_feats_imp].index.values

xgb1 = XGBClassifier(**xgb_model.get_params()).fit(x_train[feats_xgb1], y_train)



### RF with RFE

In [ ]:
%%time

rfe_rf1 = RFECV(
    rf_model,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=2),
    scoring='roc_auc', min_features_to_select=1
)
rfe_rf1.fit(x_train, y_train)


In [ ]:
_feats_mask = pd.Series(
    rfe_rf1.estimator_.feature_importances_, index=rfe_rf1.get_feature_names_out()
) > 0
feats_rf1 = _feats_mask[_feats_mask].index.values

rf1 = RandomForestClassifier(**rf_model.get_params()).fit(x_train[feats_rf1], y_train)


## Caprini Score

In [ ]:
def map_caprini_score(x):
    return 11 if x >= 11 else x


caprini_score_train = dataset_4d_caprini_dict['score_data'].loc[train_idx, 'score'].rename('Caprini')
caprini_score_test = dataset_4d_caprini_dict['score_data'].loc[test_idx, 'score'].rename('Caprini')
caprini_mapped_score_train = caprini_score_train.apply(map_caprini_score)
caprini_mapped_score_test = caprini_score_test.apply(map_caprini_score)


In [ ]:
caprini_lr = LogisticRegression(penalty='none').fit(caprini_mapped_score_train.values.reshape([-1, 1]), y_train.values)
caprini_scaled_score_train = pd.Series(
    caprini_lr.predict_proba(caprini_mapped_score_train.values.reshape([-1, 1]))[:, 1],
    index=caprini_mapped_score_train.index, name=caprini_mapped_score_train.name
)
caprini_scaled_score_test = pd.Series(
    caprini_lr.predict_proba(caprini_mapped_score_test.values.reshape([-1, 1]))[:, 1],
    index=caprini_mapped_score_test.index, name=caprini_mapped_score_test.name
)


# Model explaination

## SHAP for RF

In [ ]:

rf_explainer1 = shap.TreeExplainer(rf1)
rf_shap_values = rf_explainer1.shap_values(x_test[feats_rf1])


## SHAP for XGB


In [ ]:

xgb_explainer1 = shap.TreeExplainer(xgb1)
xgb_shap_values1 = xgb_explainer1.shap_values(x_test[feats_xgb1])


# Calibration

Calibrator for XGB, RF

In [ ]:
"""
Setup: isotonic, ensemble=True, cv without shuffle
"""

xgb1_calibrator = CalibratedClassifierCV(
    xgb1, method='isotonic', ensemble=True, cv=5  # StratifiedKFold(n_splits=5, shuffle=False)
).fit(*shuffle(x_train[feats_xgb1], y_train, random_state=0))

xgb_calib_pred_inner_test1 = xgb1_calibrator.predict_proba(x_test[feats_xgb1])[:, 1].flatten()
xgb_calib_pred_train1 = xgb1_calibrator.predict_proba(x_train[feats_xgb1])[:, 1].flatten()


In [ ]:
# rf1 
rf1_calibrator = CalibratedClassifierCV(
    rf1, method='isotonic', ensemble=True, cv=10  # StratifiedKFold(n_splits=5, shuffle=False)
).fit(*shuffle(x_train[feats_rf1], y_train, random_state=0))

rf_calib_pred_train1 = rf1_calibrator.predict_proba(x_train[feats_rf1])[:, 1].flatten()
rf_calib_pred_inner_test1 = rf1_calibrator.predict_proba(x_test[feats_rf1])[:, 1].flatten()


# Model Summary

## Prediction results

In [ ]:

_dataset_idx = y_train.index

_lr_pred = pd.Series(
    lr0.predict_proba(x_train[feats_lasso])[:, 1].flatten(), 
    index=_dataset_idx, name='LR'
)

_xgb_calib_pred = pd.Series(
    xgb1_calibrator.predict_proba(x_train[feats_xgb1])[:, 1].flatten(), 
    index=_dataset_idx, name='XGB_Calib'
)

_rf_calib_pred = pd.Series(
    rf1_calibrator.predict_proba(x_train[feats_rf1])[:, 1].flatten(),
    index=_dataset_idx, name='RF_Calib'
)

predict_results_train = pd.concat(
    [
        y_train, caprini_score_train, caprini_scaled_score_train.rename('Caprini_Calib'), 
        _lr_pred, _xgb_calib_pred, _rf_calib_pred
    ],
    axis=1
)


In [ ]:

_dataset_idx = y_test.index

_lr_pred = pd.Series(
    lr0.predict_proba(x_test[feats_lasso])[:, 1].flatten(), 
    index=_dataset_idx, name='LR'
)

_xgb_calib_pred = pd.Series(
    xgb1_calibrator.predict_proba(x_test[feats_xgb1])[:, 1].flatten(), 
    index=_dataset_idx, name='XGB_Calib'
)

_rf_calib_pred = pd.Series(
    rf1_calibrator.predict_proba(x_test[feats_rf1])[:, 1].flatten(),
    index=_dataset_idx, name='RF_Calib'
)

predict_results_test = pd.concat(
    [
        y_test, caprini_score_test, caprini_scaled_score_test.rename('Caprini_Calib'), 
        _lr_pred, _xgb_calib_pred, _rf_calib_pred
    ],
    axis=1
)


## Model metrics

In [ ]:
model_list = list(predict_results_test.columns[1:])

In [ ]:
# test_roc_details_dict: model -> data.frame of fpr, tpr, threshold

test_auc_dict = {
    _model: metrics.roc_auc_score(
        predict_results_test.label, predict_results_test[_model]
    )
    for _model in model_list
}

test_roc_details_dict = dict()
for _model in model_list:
    fpr, tpr, thre = metrics.roc_curve(
        predict_results_test.label, predict_results_test[_model]
    )
    test_roc_details_dict[_model] = pd.DataFrame(
        {'fpr': fpr, 'tpr': tpr, 'threshold': thre}
    )


In [ ]:
# train_roc_details_dict: model -> data.frame of fpr, tpr, threshold

train_auc_dict = {
    _model: metrics.roc_auc_score(
        predict_results_train.label, predict_results_train[_model]
    )
    for _model in model_list
}

train_roc_details_dict = dict()
for _model in model_list:
    _fpr, _tpr, _thre = metrics.roc_curve(
        predict_results_train.label, predict_results_train[_model]
    )
    train_roc_details_dict[_model] = pd.DataFrame(
        {'fpr': _fpr, 'tpr': _tpr, 'threshold': _thre}
    )


In [ ]:
# classif_metric_details: model -> data.frame of some common metrics w.r.t threshold

classif_metric_details = dict()
classif_metric_headings = [
    'threshold',
    'TN', 'FP', 'FN', 'TP',
    'Sensitivity', 'Specificity', 'Precision', 'NPV', 'F1', 'F1(neg)', 'MCC', 'Youden', 'Gmean'
]

for _model in model_list:
    _metric_details_list = []
    for _thre in test_roc_details_dict[_model]['threshold'].values[-2:0:-1]:
        _y_true = predict_results_test.label
        _y_pred = (predict_results_test[_model] >= _thre).astype(float)
        _tn, _fp, _fn, _tp = metrics.confusion_matrix(_y_true, _y_pred).ravel()
        _p, _r, _f, _s = metrics.precision_recall_fscore_support(_y_true, _y_pred)
        _mcc = metrics.matthews_corrcoef(_y_true, _y_pred)

        _metric_details_list.append(
            [
                _thre,
                _tn, _fp, _fn, _tp,
                _r[1], _r[0], _p[1], _p[0],
                _f[1], _f[0], _mcc,
                _r[1] + _r[0] - 1,
                np.sqrt(_r[1] * _r[0])
            ]
        )
    classif_metric_details[_model] = pd.DataFrame(
        _metric_details_list,
        columns=classif_metric_headings
    )


In [ ]:
# classif_metric_details_train: model -> data.frame of some common metrics w.r.t threshold

classif_metric_details_train = dict()

for _model in model_list:
    _metric_details_list = []
    for _thre in train_roc_details_dict[_model]['threshold'].values[-2:0:-1]:
        _y_true = predict_results_train.label
        _y_pred = (predict_results_train[_model] >= _thre).astype(float)
        _tn, _fp, _fn, _tp = metrics.confusion_matrix(_y_true, _y_pred).ravel()
        _p, _r, _f, _s = metrics.precision_recall_fscore_support(_y_true, _y_pred, zero_division=0)
        _mcc = metrics.matthews_corrcoef(_y_true, _y_pred)

        _metric_details_list.append(
            [
                _thre,
                _tn, _fp, _fn, _tp,
                _r[1], _r[0], _p[1], _p[0],
                _f[1], _f[0], _mcc,
                _r[1] + _r[0] - 1,
                np.sqrt(_r[1] * _r[0])
            ]
        )
    classif_metric_details_train[_model] = pd.DataFrame(
        _metric_details_list,
        columns=classif_metric_headings
    )


## Risk stratification
### Caprini score

In [ ]:
"""
Caprini recall: test
"""
classif_metric_caprini_df = classif_metric_details['Caprini']

thres_strat = [
    (
        'Sensitivity',
        classif_metric_caprini_df.loc[classif_metric_caprini_df['threshold'] == 3, 'Sensitivity'].iloc[0]
    ),
    (
        'Specificity',
        classif_metric_caprini_df.loc[classif_metric_caprini_df['threshold'] == 5, 'Specificity'].iloc[0])
]



In [ ]:
"""
Caprini recall: train
"""
classif_metric_caprini_df_train = classif_metric_details_train['Caprini']

thres_strat_train = [
    (
        'Sensitivity',
        classif_metric_caprini_df_train.loc[classif_metric_caprini_df_train['threshold'] == 3, 'Sensitivity'].iloc[0]
    )
,
    (
        'Specificity',
        classif_metric_caprini_df_train.loc[classif_metric_caprini_df_train['threshold'] == 5, 'Specificity'].iloc[0])
]


In [ ]:
_dev_idx = task_4d_caprini_df.loc[task_4d_caprini_df.dataset.isin(('train', 'inner_test'))].index
_train_idx = task_4d_caprini_df.loc[task_4d_caprini_df.dataset.isin(('train',))].index
_test_idx = task_4d_caprini_df.loc[task_4d_caprini_df.dataset.isin(('inner_test',))].index

caprini_level = dataset_4d_caprini_dict['score_data'].loc[_test_idx, 'score_level']
caprini_level_train = dataset_4d_caprini_dict['score_data'].loc[_train_idx, 'score_level']

y_test = dataset_4d_caprini_dict['y_data'].loc[_test_idx]
y_train = dataset_4d_caprini_dict['y_data'].loc[_train_idx]

prec_strat_caprini = pd.concat(
   [caprini_level, y_test],
   axis=1
).groupby('score_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(_test_idx),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)



In [ ]:

prec_strat_caprini_train = pd.concat(
   [caprini_level_train, y_train],
   axis=1
).groupby('score_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(_train_idx),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)


### XGB stratification

In [ ]:
def search_optimal_threshold(classif_metric_df, metric_name, metric_value, tol=1e-3, metric_backup='Youden'):
    df1 = classif_metric_df.loc[classif_metric_df[metric_name] >= metric_value - tol]
    df2 = df1.loc[df1[metric_name] == np.min(df1[metric_name])]
    if len(df2) > 1:
        idx_optimal = df2[metric_backup].idxmax()
    else:
        idx_optimal = df2.index[0]
    return classif_metric_df.loc[idx_optimal, 'threshold']

In [ ]:
# xgb calibration threshold: train
tol_value = 1e-3

classif_metric_xgb_calib_train_df = classif_metric_details_train['XGB_Calib']

thres_strat_xgb_calib_train = [
    search_optimal_threshold(classif_metric_xgb_calib_train_df, _metric, _val, tol_value)
    for _metric, _val in thres_strat_train
]

xgb_calib_level_train = pd.cut(
    predict_results_train['XGB_Calib'],
    bins=[-1, thres_strat_xgb_calib_train[0], thres_strat_xgb_calib_train[1], 2],
    right=False,
    labels=['Low', 'Median', 'High']
)

prec_strat_xgb_calib_train = pd.DataFrame(
    {'label': predict_results_train.label, 'XGB_Calib_level': xgb_calib_level_train}
).groupby('XGB_Calib_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(predict_results_train),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)
print(thres_strat_xgb_calib_train)
prec_strat_xgb_calib_train

# [0.07091680131852626, 0.10059409588575363]

In [ ]:
# xgb calibration threshold: test using train threshold

xgb_calib_level2 = pd.cut(
    predict_results_test['XGB_Calib'],
    bins=[-1, thres_strat_xgb_calib_train[0], thres_strat_xgb_calib_train[1], 2],
    right=False,
    labels=['Low', 'Median', 'High']
)

prec_strat_xgb_calib2 = pd.DataFrame(
    {'label': predict_results_test.label, 'XGB_Calib_level': xgb_calib_level2}
).groupby('XGB_Calib_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(predict_results_test),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)
# print(thres_strat_xgb_calib)
prec_strat_xgb_calib2



### RF stratification

In [ ]:
# rf calibration threshold: train

classif_metric_rf_calib_train_df = classif_metric_details_train['RF_Calib']
thres_strat_rf_calib_train = [
    search_optimal_threshold(classif_metric_rf_calib_train_df, _metric, _val, tol_value)
    for _metric, _val in thres_strat_train
]

rf_calib_level_train = pd.cut(
    predict_results_train['RF_Calib'],
    bins=[-1, thres_strat_rf_calib_train[0], thres_strat_rf_calib_train[1], 2],
    right=False,
    labels=['Low', 'Median', 'High']
)

prec_strat_rf_calib_train = pd.DataFrame(
    {'label': predict_results_train.label, 'RF_Calib_level': rf_calib_level_train}
).groupby('RF_Calib_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(predict_results_train),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)
print(thres_strat_rf_calib_train)
prec_strat_rf_calib_train


In [ ]:
# rf calibration threshold: test using train threshold

rf_calib_level2 = pd.cut(
    predict_results_test['RF_Calib'],
    bins=[-1, thres_strat_rf_calib_train[0], thres_strat_rf_calib_train[1], 2],
    right=False,
    labels=['Low', 'Median', 'High']
)

prec_strat_rf_calib2 = pd.DataFrame(
    {'label': predict_results_test.label, 'RF_Calib_level': rf_calib_level2}
).groupby('RF_Calib_level').apply(
    lambda x: pd.Series(
        {
            'pos_rate': x['label'].sum() / len(x),
            'level_count': len(x),
            'level_proportion': len(x) / len(predict_results_test),
            'pos_count': x['label'].sum(),
            'neg_count': len(x) - x['label'].sum()
        }
    )
)
prec_strat_rf_calib2



In [ ]:
# test using train cutoff
risk_strat_test = pd.concat([prec_strat_caprini, prec_strat_rf_calib2, prec_strat_xgb_calib2])
risk_strat_test['model'] = ['Caprini'] * 3 + ['RandomForest'] * 3 + ['XGBoost'] * 3



In [ ]:
risk_strat_train = pd.concat([prec_strat_caprini_train, prec_strat_rf_calib_train, prec_strat_xgb_calib_train])
risk_strat_train['model'] = ['Caprini'] * 3 + ['RandomForest'] * 3 + ['XGBoost'] * 3


# Save results

In [ ]:
save_pickle(
    task_4d_caprini_df.loc[task_4d_caprini_df.dataset.isin(['train', 'inner_test'])], 
    os.path.join(save_dir, 'task_4d_caprini_df.pkl')
)

In [ ]:
# save lr
save_pickle(
    {
        'features_lasso': feats_lasso,
        'lasso': lasso_model,
        'lr_by_lasso': lr0
    },
    file=os.path.join(save_dir, 'lr', 'lr_components.pkl')
)


In [ ]:
# save xgboost

save_pickle(
    {
        'features_xgb1': feats_xgb1,
        'raw_xgb': xgb_model,
        'xgb1': xgb1,
        'xgb1_calibrator': xgb1_calibrator
    },
    file=os.path.join(save_dir, 'xgboost', 'xgb_components.pkl')
)

In [ ]:
# save rf

save_pickle(
    {
        'features_rf1': feats_rf1,
        'raw_rf': rf_model,
        'rf1': rf1,
        'rf1_calibrator': rf1_calibrator,
    },
    file=os.path.join(save_dir, 'rf', 'rf_components.pkl')
)

In [ ]:
# save prediction values

save_pickle(
    {
        'train': predict_results_train,
        'test': predict_results_test
    },
    file=os.path.join(save_dir, 'prediction.pkl')
)



In [ ]:
# save shap explainer
save_pickle(xgb_explainer1, file=os.path.join(save_dir, 'xgboost', 'shap_explainer.pkl'))
save_pickle(rf_explainer1, file=os.path.join(save_dir, 'rf', 'shap_explainer.pkl'))

In [ ]:
# save stratification
save_dataframe_dict(
    {'train': risk_strat_train, 'test': risk_strat_test},
    file=os.path.join(save_dir, 'risk_stratification.xlsx')
)
save_pickle(
    {'train': risk_strat_train, 'test': risk_strat_test},
    file=os.path.join(save_dir, 'risk_stratification.pkl')
)

strat_level_details = {
    'train': pd.DataFrame(
        {
            'Caprini': caprini_level_train,
            'XGB_Calib': xgb_calib_level_train,
            'RF_Calib': rf_calib_level_train
        }
    ),
    'test': pd.DataFrame(
        {
            'Caprini': caprini_level,
            'XGB_Calib': xgb_calib_level2,
            'RF_Calib': rf_calib_level2
        }
    ),
    'cutoff': pd.DataFrame(
        {
            'Caprini': [3, 5],
            'XGB_Calib': thres_strat_xgb_calib_train,
            'RF_Calib': thres_strat_rf_calib_train
        }
    )
}

save_dataframe_dict(
    strat_level_details,
    file=os.path.join(save_dir, 'risk_stratification_details.xlsx')
)
save_pickle(
    strat_level_details,
    file=os.path.join(save_dir, 'risk_stratification_details.pkl')
)
